# Read SpaceX Launch Data from https://api.spacexdata.com/v4

Documentation of the SpaceX API can be found @ https://github.com/r-spacex/SpaceX-API/tree/master/docs

In [ ]:
import requests

# Pick a dataset and the appropriate url (will require url+"<data>" in other calls to pull specific data)
url = "https://api.spacexdata.com/v4/"
# Get launch data
launches = requests.get(url+"launches").json()
# Get payload data
payloads = requests.get(url+"payloads").json()
# Get rockets data
rockets = requests.get(url+"rockets").json()

In [ ]:
print(f"There are {len(launches)} launches")
print(f"There are {len(payloads)} payloads")
print(f"There are {len(rockets)} rockets")

In [ ]:
# for launch in launches:
#   print(launch.get('name'), launch.get("id"), launch.get('date_utc'), launch.get("rocket"))

In [ ]:
# for rocket in rockets:
#   print(rocket.get("name"), rocket.get("id"), rocket.get("type"), rocket.get("cost_per_launch"))

In [ ]:
# for payload in payloads:
#   print(payload.get("name"), payload.get("id"), payload.get("type"), payload.get("launch"))

In [101]:
import os
import sqlite3

# Delete the old database file if it exists
if os.path.exists("spacex.db"):
    os.remove("spacex.db")
    print("spacex.db database deleted")

spacex.db database deleted


In [104]:
# Create a connection (this creates spacex.db in your current Colab environment)
conn = sqlite3.connect("spacex.db")

# Create a cursor
cur = conn.cursor()

schema = """
CREATE TABLE IF NOT EXISTS rockets (
    id TEXT PRIMARY KEY,
    name TEXT,
    type TEXT,
    cost_per_launch INTEGER

);

CREATE TABLE IF NOT EXISTS launches (
    id TEXT PRIMARY KEY,
    name TEXT,
    date_utc TEXT,
    rocket_id TEXT
);

CREATE TABLE IF NOT EXISTS payloads (
    id TEXT PRIMARY KEY,
    name TEXT,
    type TEXT,
    mass_kg INTEGER,
    launch_id TEXT
);
"""

cur.executescript(schema)
conn.commit()
print("Database schema created")

Database schema created


In [105]:
# Insert data from api into rockets table, with only the columns we want
# ignore preventing pk conflicts
for r in rockets:
    cur.execute("""
        INSERT OR IGNORE INTO rockets
        (id, name, type, cost_per_launch)
        VALUES (?, ?, ?, ?)
    """, (
        r["id"], r["name"], r["type"], r["cost_per_launch"]
    ))

conn.commit()
print("Rockets data inserted")

Rockets data inserted


In [106]:
# Insert data from api into launches table, with only the columns we want
# ignore preventing pk conflicts
for l in launches:
    cur.execute("""
        INSERT OR IGNORE INTO launches
        (id, name, date_utc, rocket_id)
        VALUES (?, ?, ?, ?)
    """, (
        l["id"], l["name"], l["date_utc"], l["rocket"]
    ))

conn.commit()
print("Launches data inserted")

Launches data inserted


In [109]:
# Insert data from api into payloads table, with only the columns we want
# ignore preventing pk conflicts
for p in payloads:
    cur.execute("""
        INSERT OR IGNORE INTO payloads
        (id, name, type, mass_kg, launch_id)
        VALUES (?, ?, ?, ?, ?)
    """, (
        p["id"], p["name"], p["type"], p["mass_kg"], p["launch"]
    ))

conn.commit()
print("Payloads data inserted")

Payloads data inserted


In [96]:
import pandas as pd

# Display table as a dataframe
df = pd.read_sql_query("SELECT * FROM launches LIMIT 5;", conn)
df

,id,name,date_utc
0,5eb87cd9ffd86e000604b32a,FalconSat,2006-03-24T22:30:00.000Z
1,5eb87cdaffd86e000604b32b,DemoSat,2007-03-21T01:10:00.000Z
2,5eb87cdbffd86e000604b32c,Trailblazer,2008-08-03T03:34:00.000Z
3,5eb87cdbffd86e000604b32d,RatSat,2008-09-28T23:15:00.000Z
4,5eb87cdcffd86e000604b32e,RazakSat,2009-07-13T03:35:00.000Z
